In [22]:
import zipfile
from PIL import Image
import cv2 as cv
import numpy as np
import pytesseract
import math
import os
import darknet


In [23]:
# Define the YOLO detector
def yolo_detector(img, net, meta):
    """
    Detect objects in the image using YOLO.
    """
    width = darknet.network_width(net)
    height = darknet.network_height(net)
    darknet_image = darknet.make_image(width, height, 3)
    frame_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    frame_resized = cv.resize(frame_rgb, (width, height), interpolation=cv.INTER_LINEAR)
    darknet.copy_image_from_bytes(darknet_image, frame_resized.tobytes())
    detections = darknet.detect_image(net, meta, darknet_image, thresh=0.5)
    darknet.free_image(darknet_image)
    return detections


In [24]:
net = darknet.load_network("/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/yolov4-tiny.cfg".encode("ascii"), "/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/yolov4-tiny.weights".encode("ascii"), 0, 1)
meta = darknet.load_meta("/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/coco.data".encode("ascii"))


AttributeError: module 'darknet' has no attribute 'load_network'

In [10]:
output_folder = '/Users/yatinpatel/Desktop/Python3-Programming-Course/output'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [11]:
def zip_images_extraction(name):
    """
    Get all the information (name, image, text) from a ZIP file.
    :input: The name of a ZIP file.
    :output: A list of dictionaries. Each dictionary contains all the information (name, image, text) of an image object.
    """
    # ZIP name
    zip_name = name

    # Output
    out = []

    # Extract all the information
    with zipfile.ZipFile(zip_name) as myzip:
        zip_infos = myzip.infolist()

        for ele in zip_infos:
            # Name
            name = ele.filename
            # Image
            img = Image.open(myzip.open(name))
            # Text
            img_strs = pytesseract.image_to_string(img.convert('L'))

            # Test if "Christopher" or "Mark" are in the text
            if "Christopher" in img_strs or "Mark" in img_strs:
                # Example of a dictionary
                my_dict = {"name": name, "img": img, "text": img_strs}
                out.append(my_dict)
    return out

In [12]:
def extract_faces(img, net, meta):
    # Extract the rectangles of the faces
    img_cv = np.array(img)
    detections = yolo_detector(img_cv, net, meta)

    # Filter detections to only keep those corresponding to "person"
    faces = [det for det in detections if det[0].decode() == "person"]

    # If no faces are detected
    if len(faces) == 0:
        return None

    # Extract faces into the list
    faces_imgs = []
    for label, confidence, bbox in faces:
        x, y, w, h = bbox
        faces_imgs.append(img.crop((x, y, x + w, y + h)))
        x, y, w, h = bbox
        x, y, w, h = int(x), int(y), int(w), int(h)  # Convert to integers
        faces_imgs.append(img.crop((x, y, x + w, y + h)))

    # Compute nrows and ncols
    ncols = 5
    nrows = math.ceil(len(faces) / ncols)

    # Create a contact sheet
    contact_sheet = Image.new(img.mode, (550, 110 * nrows))
    x, y = (0, 0)

    for face in faces_imgs:
        face.thumbnail((110, 110))
        contact_sheet.paste(face, (x, y))

        if x + 110 == contact_sheet.width:
            x = 0
            y += 110
        else:
            x += 110

    return contact_sheet

In [13]:
def value_search(value, zip_name, thresh):
    if zip_name == "small_img.zip":
        ref_imgs = small_imgs
    else:
        ref_imgs = big_imgs

    for ele in ref_imgs:
        # Test if value is in the text
        if value in ele["text"]:
            # Print out the name of the figure
            print("Results found in file {}".format(ele["name"]))

            # Index out the faces
            img = ele["img"]
            contact_sheet = extract_faces(img, thresh)
            if contact_sheet is not None:
                # Save the contact sheet
                contact_sheet.save(os.path.join(output_folder, ele["name"].split('.')[0] + '.png'))
                display(contact_sheet)
            else:
                print("But there were no faces in that file")

In [14]:
# Extract all the information related to small_img.zip and images.zip
small_imgs = zip_images_extraction("small_img.zip")
big_imgs = zip_images_extraction("images.zip")

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
# Reproduce the search for "Christopher"
value = "Christopher"
zip_name = "small_img.zip"
value_search(value, zip_name, thresh=0.5)

In [ ]:
# Reproduce the search for "Mark"
value = "Mark"
zip_name = "images.zip"
value_search(value, zip_name, thresh=0.5)

In [25]:
import darknet

try:
    # Load YOLOv4-tiny model
    net = darknet.load_network("/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/yolov4-tiny.cfg".encode("ascii"),
                               "/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/yolov4-tiny.weights".encode("ascii"), 0, 1)

    # Load COCO dataset metadata
    meta = darknet.load_meta("/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/coco.data".encode("ascii"))

    # Continue with your code...
except Exception as e:
    print("An error occurred:", str(e))

An error occurred: module 'darknet' has no attribute 'load_network'


In [ ]:
import darknet

try:
    # Load YOLOv4-tiny model
    net = darknet.load_net_custom("/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/yolov4-tiny.cfg".encode("ascii"),
                                  "/Users/yatinpatel/Desktop/Python3-Programming-Course/yolo/yolov4-tiny.weights".encode("ascii"), 0, 1)

    # Load COCO dataset metadata
    meta = darknet.load_meta("/path/to/coco.data".encode("ascii"))

    # Continue with your code...
except Exception as e:
    print("An error occurred:", str(e))
